## [412周赛](https://leetcode.cn/contest/weekly-contest-412/)

### [统计近似相等数对 II](https://leetcode.cn/problems/count-almost-equal-pairs-ii/description/)
### 题目
给你一个正整数数组nums。如果我们执行以下操作至多两次可以让两个整数x和y相等，那么我们称这个数对是近似相等的：
- 选择x或者y之一，将这个数字中的两个数位交换。

请你返回nums中，下标i和j满足i < j且nums[i]和nums[j]近似相等的数对数目。注意执行操作后得到的整数可以有前导0 。


### 分析
- 枚举右, 维护左: 向右边遍历数组时使用哈希表维护index左边数据的状态
- 由于可以存在前导0, 因此我们有两种做法, 一种是在每个数上加上前导0, 还有一种是交换时我们只选择长度大的进行交换(因为长度大的可以通过与0换位置得到长度小的)。这里我们为了方便选择第二种处理方法(因为每个数加上前导0会对后面造成很多麻烦), 为了能在维护左边数据状态时一定是左边数的长度小于右边的, 因此我们需要根据长度将nums进行排序
- 此处我们维护左边数据状态的方法就是, 记录数字出现次数. 当我们遍历到numj时, 将numj经过之多两次交换能得到哪些数字用集合(用于去重)记录下来, 然后遍历集合, 看看左边的数据状态中这些数字出现了多少次, 我们的答案就需要+多少次
- 我们可以使用暴力的做法将至多交换两次可以得到的数据放到集合中(暴力做法中注意要复原现场)


In [ ]:
import collections

def process(nums):
    nums.sort(key=lambda x: len(str(x)))
    ans = 0
    cnt = collections.defaultdict(int)
    for num in nums:
        st = {num}  # 什么都不交换
        s = list(str(num))  # 得到数值列表, 方便交换
        m = len(s)
        # 暴力交换
        for i in range(m):
            for j in range(i+1, m):
                s[i], s[j] = s[j], s[i]   # 第一次交换
                st.add(int("".join(s)))

                for p in range(i+1, m):   # 这里从i+1开始是为了避免和第一次交换重复(不需要再从0开始交换)
                    for q in range(p+1, m):
                        s[p], s[q] = s[q], s[p]  # 第二次交换
                        st.add(int("".join(s)))
                        s[p], s[q] = s[q], s[p]  # 注意复原
                s[i], s[j] = s[j], s[i]  # 注意复原
        # 遍历左边数据状态得到答案
        ans += sum(cnt[v] for v in st)
        # 更改左边数据状态
        cnt[num] += 1   
    return ans
        

            


### [K 次乘运算后的最终数组 II](https://leetcode.cn/problems/final-array-state-after-k-multiplication-operations-ii/description/)
### 题目
给你一个整数数组nums ，一个整数k和一个整数multiplier 。你需要对nums执行k次操作，每次操作中：
- 找到 nums 中的 最小 值 x ，如果存在多个最小值，选择最 前面 的一个。
- 将 x 替换为 x * multiplier 。

k次操作以后，你需要将nums中每一个数值对 10^9 + 7 取余。请你返回最终的nums数组。(注意这里的k非常大)

### 分析
- 我们可以使用最小堆的方式, 这样可以更快的得到最小的那个数(使用heapq.heapreplace可以更高效的替换堆中最小元素). 但由于这里的K非常大, 因此我们要想办法让操作于K无关。
- 经过分析可以知道两个结论:
    - 1、当x,y,z很接近时, 一定是从小到大进行依次乘以m的, 根据这个规律我们可以直接得到每个元素还需要进行多少次的乘法: 如果排序后i < k%n, 则还需要执行 k//n + 1次; 如果i >= k%n, 则还需要执行k//n 次的乘法(因为此处i从0开始, 所以当k%n=0时是不需要多执行一次的)
    - 2、对于原始数组, 我们可以先使用循环最小堆的方式, 不断将数组中每个元素乘以m, 使得他们相互靠近, 当最后出堆的是num原始中的最大值时, 此时数组就满足了结论1的先决条件: 各数组都非常靠近
- 需要注意到一些特殊情况, 如果k比较小时可能在循环最小堆的时候就已经解决问题了, 因此循环最小堆的循环条件是k > 0 ; 当multiplier为1时, 是不可能退出得了循环最小堆的(因为最小是*1是不可能靠近最大值的)

In [2]:
import heapq

def process(nums, k, multiplier):
    if multiplier == 1:
        return nums

    MOD = 1_000_000_000 + 7
    n = len(nums)
    mx = max(nums)
    h = [(num, i) for i, num in enumerate(nums)]  # 需要保留坐标
    heapq.heapify(h)

    # 在此处可以先不进行nums的变换
    while k > 0:
        num, i = h[0]  # 拿到堆中最小值
        heapq.heapreplace(h, (num*multiplier, i))   # 先pop再push, 更高效
        k -= 1         # 注意此处要将k减小
        if num == mx:  # 如果出堆的是原始数组中的最小值, 那么说明此时已经满足很靠近的条件了
            break
    
    for i in range(n):  # 这里的i表示就是此时num中数组排续的位次
        num, j = heapq.heappop(h)
        nums[j] = num * pow(multiplier, k//n + (i < k%n), MOD) % MOD   # 这里使用int + bool巧妙地进行了if操作
    return nums

## [416周赛](https://leetcode.cn/contest/weekly-contest-416/)

### [移山所需的最少秒数](https://leetcode.cn/problems/minimum-number-of-seconds-to-make-mountain-height-zero/description/)
### 题目
给你一个整数 mountainHeight 表示山的高度。同时给你一个整数数组 workerTimes，表示工人们的工作时间（单位：秒）。对于工人i: 把山的高度降低x，需要花费workerTimes[i] + workerTimes[i] * 2 +...+ workerTimes[i] * x秒。例如:
- 山的高度降低 1，需要 workerTimes[i] 秒。
- 山的高度降低 2，需要 workerTimes[i] + workerTimes[i] * 2 秒，依此类推。

返回一个整数，表示工人们使山的高度降低到0所需的最少秒数。注意: 工人们是同时进行工作的。

### 分析
- 由于工人们是同时进行工作, 所以得到每个工人工作时间列表后需要求最大值. 我们可以使用模拟堆的方式, 每次选一个「工作后总用时」最短的工人，把山的高度降低 1。因此我们的堆中元素为(工作后该员工总用时, 下一次工作的增量时间, 工人基础时间)
- 每次工人工作后维护当前最多用时

In [ ]:
import heapq
def process(mountainHeight: int, workerTimes):
    cost = [(num, 2*num, num) for num in workerTimes]
    heapq.heapify(cost)

    ans = 0
    for _ in range(mountainHeight):  # 循环山的高度, 每次挑选「工作后总用时」最短的工人
        time, delta, base = heapq.heappop(cost)
        ans = max(0, time)
        heapq.heappush(cost, (time+delta, delta+base, base))
    return ans

mountainHeight = 5
workerTimes = [1, 7]
process(mountainHeight, workerTimes)

### [统计重新排列后包含另一个字符串的子字符串数目 II](https://leetcode.cn/problems/count-substrings-that-can-be-rearranged-to-contain-a-string-ii/description/)
### 题目
给你两个字符串word1和 word2。如果一个字符串x重新排列后，word2是重排字符串的前缀，那么我们称字符串x是合法的。请你返回word1中合法子字符串的数目。

注意: 必须实现一个线性复杂度的解法。(内存要小)

### 分析
- 


In [ ]:
def char_count(s):
    result = {}
    for char in s:
        if char in result:
            result[char] += 1
        else:
            result[char] = 1
    return result

def process(word1, word2):
    left = []
    word2_dict = char_count(word2)
    you = len(word2_dict)
    
    for index, char in enumerate(word1):
        if char in word2_dict:
            left.append(index)
            word2_dict[char] -= 1
            if word2_dict[char] == 0:
                you -= 1
                if you == 0:
                    break
    if you != 0:
        return 0
    right_index = index
    left_index = left.pop(0)
    ans = left_index + len(word1) - right_index
    print("开始:",left_index, right_index, left)
    for index in range(right_index+1, len(word1)):
        
        if word1[index] in word2_dict:
            left.append(index)
            if word1[index] == word1[left_index]:
                
                left_index = left.pop(0)
                ans += left_index
                print("更新:",left_index, index, left)
    return ans


word1 = "abcabc"
word2 = "abc"
process(word1, word2)

### []()
### 题目



### 分析


In [ ]:
a = [1,23]
a.pop(0)